In [1]:
import numpy as np
import pandas as pd

In [2]:
#日期，业务种类，重量，揽投机构，道段(揽收只有业务员信息，没有道段故不收入)

#读取揽收数据
data_lanshou = pd.read_csv('江门揽收数据.csv', encoding='gb18030', usecols=[1,3,6,8])
data_lanshou = data_lanshou[['D_CLCT_DATE','V_BUSI_TYPE', 'F_WEIGH_WEIGHT', 'V_CLCT_ORG_CODE']]#调整一下顺序
data_lanshou['D_CLCT_DATE'] = pd.to_datetime(data_lanshou['D_CLCT_DATE'])
#加入经纬度
data_lanshou_ = pd.read_csv('coord/lanshou.csv', encoding='gb18030', usecols=[1,2])
data_lanshou['lng'] = data_lanshou_['lng']
data_lanshou['lat'] = data_lanshou_['lat']

#读取投递数据1
data_toudi21 = pd.read_csv('toudi_21.csv', encoding='gb18030', usecols=[0,2,3,4,7])
data_toudi21['道段'] = data_toudi21['道段'].str.split(' ', expand=True)[0]
data_toudi21['投递日期'] = pd.to_datetime(data_toudi21['投递日期'])
#加入经纬度
data_toudi21_  = pd.read_csv('coord/toudi_21.csv', encoding='gb18030', usecols=[1,2])
data_toudi21['lng'] = data_toudi21_['lng']
data_toudi21['lat'] = data_toudi21_['lat']

#读取投递数据2
data_toudi22 = pd.read_csv('toudi_21.csv', encoding='gb18030', usecols=[0,2,3,4,7])
data_toudi22['道段'] = data_toudi22['道段'].str.split(' ', expand=True)[0]
data_toudi22['投递日期'] = pd.to_datetime(data_toudi22['投递日期'])
#加入经纬度
data_toudi22_  = pd.read_csv('coord/toudi_21.csv', encoding='gb18030', usecols=[1,2])
data_toudi22['lng'] = data_toudi22_['lng']
data_toudi22['lat'] = data_toudi22_['lat']

#读取投递数据3
data_toudi206 = pd.read_csv('toudi_21.csv', encoding='gb18030', usecols=[0,2,3,4,7])
data_toudi206['道段'] = data_toudi206['道段'].str.split(' ', expand=True)[0]
data_toudi206['投递日期'] = pd.to_datetime(data_toudi206['投递日期'])
#加入经纬度
data_toudi206_  = pd.read_csv('coord/toudi_21.csv', encoding='gb18030', usecols=[1,2])
data_toudi206['lng'] = data_toudi206_['lng']
data_toudi206['lat'] = data_toudi206_['lat']

In [3]:
#exclude为不需要的日期列表，下述代码将周末的数据排除
exclude = ["2020-08-01","2020-08-02","2020-08-08","2020-08-09","2020-08-15","2020-08-16","2020-08-22","2020-08-23","2020-08-29","2020-08-30"]

bools = []
for row in data_lanshou.values:
    bools.append(str(row[0].date()) not in exclude)
data_lanshou = data_lanshou.loc[bools]

bools = []
for row in data_toudi21.values:
    bools.append(str(row[0].date()) not in exclude)
data_toudi21 = data_toudi21.loc[bools]

bools = []
for row in data_toudi22.values:
    bools.append(str(row[0].date()) not in exclude)
data_toudi22 = data_toudi22.loc[bools]
    
bools = []
for row in data_toudi206.values:
    bools.append(str(row[0].date()) not in exclude)
data_toudi206 = data_toudi206.loc[bools]

In [46]:
#揽收默认的列属性不太一样，改成与投递一致
new_cols = ["投递日期","业务种类","重量","投递机构","lng","lat"]
data_lanshou.columns = new_cols

In [58]:
lanshou = []
toudi21 = []
toudi22 = []
toudi206 = []

departments = pd.read_excel('江门补充资料/江门机构.xlsx', usecols=[3])
for department in departments.values:
    batch = data_lanshou.loc[data_lanshou['投递机构']==int(department)]
    lanshou.append(batch)
    batch = data_toudi21.loc[data_toudi21['投递机构']==int(department)]
    toudi21.append(batch)
    batch = data_toudi22.loc[data_toudi22['投递机构']==int(department)]
    toudi22.append(batch)
    batch = data_toudi206.loc[data_toudi206['投递机构']==int(department)]
    toudi206.append(batch)

In [65]:
print(toudi21)

[                      投递日期 业务种类     重量      投递机构   道段         lng        lat
52343  2020-08-03 09:07:35   快包  1.000  52900009  T33  113.070938  22.590227
52516  2020-08-03 09:11:49   标快  1.450  52900009   18  113.075926  22.573499
52525  2020-08-03 09:12:23   快包  0.500  52900009   03  113.087034  22.575283
52684  2020-08-03 09:15:06   标快  0.200  52900009   02  113.075836  22.584081
52702  2020-08-03 09:15:49   快包  1.000  52900009   03  113.087034  22.575283
...                    ...  ...    ...       ...  ...         ...        ...
992546 2020-08-31 18:39:33   快包  1.403  52900009   76  113.074693  22.591838
992785 2020-08-31 20:32:36   标快  0.100  52900009  T31  113.076125  22.591575
992942 2020-08-31 18:05:28  NaN    NaN  52900009   23         NaN        NaN
993433 2020-08-31 18:05:46  NaN    NaN  52900009   23         NaN        NaN
993496 2020-08-31 18:18:07   快包  0.690  52900009   75  113.084324  22.583424

[19282 rows x 7 columns],                       投递日期 业务种类     重量      投递机构